In [ ]:
# Import all the modules

In [ ]:
# Load the dataset

In [ ]:
# Add new features
# Molecular weight
# Count the occurrences of each amino acid
# Count the total number of atoms, including carbon (C), hydrogen (H), oxygen (O), nitrogen (N), sulfur (S), etc.
# Count the number of aromatic rings
# logP
# ph, charge?
# number of hydrogen, covalent bond, diff bod


In [ ]:
# Preprocess the dataset

In [ ]:
# Split the dataset 

In [ ]:
# Build the model

In [ ]:
# Test the accuracy of the set